# 製作詞頻矩陣

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import jieba
from time import sleep
import re

In [2]:
April_Data = pd.read_csv("April_Gossiping_Data.csv")
April_Data.head()

,Unnamed: 0,Date,Push,Title,Author,Link,Push_Count,Content
0,0,4/01,NaN,[問卦] 八卦板取年號該取啥,med5566,/bbs/Gossiping/M.1554113138.A.5BC.html,17,各位八卦鄉民們晚安\n\n年號通常有皇帝對於未來期許的意味\n\n如果八卦板也來年號紀元\n...
1,1,4/01,10,[新聞] 高雄簽13億無法律效力MOU 陳吉仲：農委,qqq87112,/bbs/Gossiping/M.1554113163.A.4CE.html,27,1.媒體來源:\n\n\n自由\n2.完整新聞標題:\n\n高雄簽13億無法律效力MOU 陳...
2,2,4/01,5,[問卦] 有沒有雨衣只是把雨水集中到鞋子裡的八卦,ken0062,/bbs/Gossiping/M.1554113195.A.80D.html,27,台灣人最愛騎機車，\n\n一遇到下雨就只能穿雨衣，\n\n但雨衣的設計根本就沒有辦法騎機車，...
3,3,4/01,5,[新聞] 紐航將開通直飛首爾航班 機上Wifi將免費,FreeWiFi,/bbs/Gossiping/M.1554113228.A.0A0.html,10,大紀元\n\n\n紐航將開通直飛首爾航班 機上Wifi將免費\n\n\n【大紀元2019年0...
4,4,4/01,NaN,[問卦] 有沒有賣國旗的外國人的八卦,WWTDD,/bbs/Gossiping/M.1554113273.A.A5E.html,8,之前有個自稱聾啞的外國人拿著國旗給路人\n路人如果伸手拿了 就會拿出張紙寫一支國旗100塊...


# 1.文章清洗

In [3]:
doc_content = list(April_Data["Content"])
doc_title = list(April_Data["Title"])


#doc_content
#doc_title


doc_title_test = doc_title[0:10000]
doc_content_test = doc_content[0:10000]

#doc_title_test
#doc_content_test

In [4]:
#停用字詞1

prog = re.compile('[\s+\.\!\/_,$%^*(+\"\'-]+|[+——→！，。？：:；“”、~@#￥%……&*（）(\d+)「」『』《》【】│]+|[0-9]+|[a-z]', re.I)

#停用字詞2

stop_words = [ ]
with open('Stop_Words.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stop_words.append(data)
stop_words[0:5]

['$', '0', '1', '2', '3']

In [5]:
#切詞

jieba.load_userdict("term_2.txt")
clean_words = [ ]

for i in range(10000):
    clean = prog.sub(' ', doc_content_test[i])
    seg_words = jieba.cut(clean , cut_all = False)
    remainder_words = list(filter(lambda a:a not in stop_words and a != "\n" , seg_words))
    
    clean_words.append(remainder_words)
    
clean_words[0:5]    
len(clean_words)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/qx/7hg_r3x1563bd0s5rys142_00000gn/T/jieba.cache
Loading model cost 1.308 seconds.
Prefix dict has been built succesfully.


10000

In [6]:
#看一下切詞結果

df = pd.DataFrame()
df["Title"] = doc_title_test
df["Seg_Content"] = clean_words

df.head()

,Title,Seg_Content
0,[問卦] 八卦板取年號該取啥,"[八卦, 鄉民們, 晚安, , 年號, 通常, 皇帝, 未來, 期許, 意味, , 八卦..."
1,[新聞] 高雄簽13億無法律效力MOU 陳吉仲：農委,"[ , , 媒體, 來源, , , 自由, , , , 完整, 新聞, 標題, ..."
2,[問卦] 有沒有雨衣只是把雨水集中到鞋子裡的八卦,"[台灣, 最愛騎, 機車, , , 遇到, 下雨, 只能, 穿, 雨衣, , , 雨..."
3,[新聞] 紐航將開通直飛首爾航班 機上Wifi將免費,"[大紀元, , 紐航, 將開, 通直, 飛首爾, 航班, , 機上, , , , ..."
4,[問卦] 有沒有賣國旗的外國人的八卦,"[之前, 有個, 自稱, 聾, 啞, 外國人, 著國旗, 給路, , 路, 伸手, , ..."


# 2.建立Topic Model

In [46]:
#!pip install gensim

In [7]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [8]:
from pprint import pprint

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(clean_words)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in clean_words]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, 
                                           random_state=100, update_every=1, chunksize=10,
                                           passes=10, alpha='symmetric', iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.017*"陳" + 0.010*"男" + 0.008*"女童" + 0.008*"乘客" + 0.007*"林" + 0.007*"男子" + '
  '0.007*"性交" + 0.006*"問卦" + 0.006*"警方" + 0.006*"林女"'),
 (1,
  '0.650*" " + 0.006*"台灣" + 0.005*"說" + 0.004*"中國" + 0.003*"美國" + 0.003*"推" + '
  '0.002*"年" + 0.002*"=" + 0.002*"新聞" + 0.002*"長"'),
 (2,
  '0.020*"柯文哲" + 0.017*"總統" + 0.016*"民進黨" + 0.011*"蔡英文" + 0.011*"初選" + '
  '0.011*"國民黨" + 0.009*"協調" + 0.007*"台北市" + 0.007*"韓市" + 0.006*"黨"'),
 (3,
  '0.027*"沒" + 0.015*"會" + 0.009*"八卦" + 0.008*"吃" + 0.008*"一個" + 0.006*"看到" + '
  '0.006*"應該" + 0.006*"是不是" + 0.005*"問題" + 0.005*"肥宅"'),
 (4,
  '0.022*"─" + 0.018*"文化" + 0.012*"軍" + 0.012*"中華" + 0.011*"北京" + 0.010*"國" + '
  '0.009*"飛彈" + 0.009*"防部" + 0.008*"兩岸" + 0.008*"戰機"')]


In [14]:
!pip install pyldavis
import gensim
from gensim import models
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.430812  0.088615       1        1  82.711197
3     -0.003102 -0.346995       2        1   9.796635
2      0.136101  0.108242       3        1   3.186307
0      0.147533  0.070945       4        1   2.388045
4      0.150280  0.079194       5        1   1.917816, topic_info=      Category          Freq Term         Total  loglift  logprob
term                                                             
0      Default  1.324388e+06       1.324388e+06  30.0000  30.0000
109    Default  7.930000e+03    沒  7.930000e+03  29.0000  29.0000
387    Default  6.735000e+03    會  6.735000e+03  28.0000  28.0000
769    Default  3.090000e+03   一個  3.090000e+03  27.0000  27.0000
1273   Default  1.592000e+03  柯文哲  1.592000e+03  26.0000  26.0000
704    Default  3.752000e+03   問題  3.752000e+03  25.0000  25.0000
3      Default  2.293000e+03   八卦  2.293000e+03  24.0000  24.0000
1950   Default  1.973000e+03    吃  1.973000e+03  23.0000  23.0000
2265   Default  1.315000e+03   總統  1.315000e+03  22.0000  22.0000
942    Default  1.452000e+03    陳  1.452000e+03  21.0000  21.0000
2602   Default  1.255000e+03  民進黨  1.255000e+03  20.0000  20.0000
658    Default  1.115000e+03    軍  1.115000e+03  19.0000  19.0000
392    Default  2.063000e+03   看到  2.063000e+03  18.0000  18.0000
7787   Default  1.017000e+03    ─  1.017000e+03  17.0000  17.0000
714    Default  2.115000e+03   應該  2.115000e+03  16.0000  16.0000
826    Default  1.474000e+03    前  1.474000e+03  15.0000  15.0000
735    Default  2.133000e+03    話  2.133000e+03  14.0000  14.0000
761    Default  1.821000e+03   有人  1.821000e+03  13.0000  13.0000
955    Default  1.556000e+03  是不是  1.556000e+03  12.0000  12.0000
2136   Default  8.500000e+02   文化  8.500000e+02  11.0000  11.0000
1089   Default  1.014000e+03   相關  1.014000e+03  10.0000  10.0000
4023   Default  8.700000e+02  蔡英文  8.700000e+02   9.0000   9.0000
4624   Default  8.640000e+02   初選  8.640000e+02   8.0000   8.0000
999    Default  2.541000e+03   可能  2.541000e+03   7.0000   7.0000
2549   Default  8.380000e+02  國民黨  8.380000e+02   6.0000   6.0000
1337   Default  1.227000e+03   肥宅  1.227000e+03   5.0000   5.0000
390    Default  1.475000e+03   發現  1.475000e+03   4.0000   4.0000
816    Default  9.450000e+02    先  9.450000e+02   3.0000   3.0000
3734   Default  1.318000e+03    死  1.318000e+03   2.0000   2.0000
6564   Default  6.990000e+02   協調  6.990000e+02   1.0000   1.0000
...        ...           ...  ...           ...      ...      ...
11765   Topic5  2.858582e+02   戰爭  2.866387e+02   3.9513  -5.1073
6029    Topic5  2.746014e+02   軍事  2.753713e+02   3.9512  -5.1474
3424    Topic5  2.382148e+02   和平  2.389840e+02   3.9508  -5.2896
11668   Topic5  2.086551e+02  菲律賓  2.094132e+02   3.9504  -5.4221
1042    Topic5  2.018002e+02   思想  2.026005e+02   3.9500  -5.4555
10387   Topic5  2.067661e+02    駐  2.076377e+02   3.9498  -5.4312
2995    Topic5  1.806019e+02   武器  1.813669e+02   3.9498  -5.5665
3003    Topic5  1.680980e+02   海峽  1.688534e+02   3.9495  -5.6382
7910    Topic5  1.715318e+02   戰略  1.723151e+02   3.9494  -5.6180
6912    Topic5  1.568683e+02  戰鬥機  1.576656e+02   3.9489  -5.7073
11670   Topic5  1.441529e+02   裝備  1.449151e+02   3.9487  -5.7919
2947    Topic5  1.322956e+02   基地  1.330555e+02   3.9483  -5.8777
3225    Topic5  1.303699e+02   解放  1.311262e+02   3.9482  -5.8924
10605   Topic5  1.283171e+02    艘  1.290750e+02   3.9481  -5.9082
1659    Topic5  1.367719e+02   民族  1.375876e+02   3.9480  -5.8444
17839   Topic5  1.418092e+02   海空  1.426553e+02   3.9480  -5.8083
11632   Topic5  1.287435e+02   潛艦  1.295260e+02   3.9479  -5.9049
618     Topic5  1.280777e+02   挑釁  1.288569e+02   3.9479  -5.9101
11637   Topic5  1.748170e+02   登陸  1.759760e+02   3.9474  -5.5990
21541   Topic5  1.116523e+02   雷達  1.124183e+02   3.9471  -6.0474
79507   Topic5  2.065253e+02  中業島  2.082298e+02   3.9458  -5.4323
4761    Topic5 